In [160]:
import pandas as pd

## 1. Reading the csv file

In [161]:
#Reading CSV file
arq = "FC_Mixer_MO.csv"
df = pd.read_csv(arq, sep=',')

## 2. Data Process

In [162]:
#Dropping the columns that we not gonna use
df = df.drop(['site', 'division', 'category', 'line', 'model_name', 'recipe_group', 'product_code'], axis=1)

In [163]:
#Dropping rows with negative values
df = df[df['char_value'] >= 0]

In [164]:
#Filtering to retrieve only the completed batches
df = df[(df['model_complete']=='COMPLETE')]

In [165]:
df.char_name.unique()

array(['non_value_operating_time', 'phase_1.Inicio.duration',
       'phase_2.Medicion.duration', 'phase_3.Recirculacion.duration',
       'phase_3.Recirculacion.Peso_avg',
       'phase_3.Recirculacion.Densidad_max',
       'phase_3.Recirculacion.Presion_avg',
       'phase_3.Recirculacion.Presion_max',
       'phase_3.Recirculacion.Visco_max', 'total_duration'], dtype=object)

In [166]:
def nova_coluna(texto, col_type='phase'):
    phase_mapping = {
        'non_value_operating_time': {'phase': 0, 'name': 'Operating'},
        'phase_1.Inicio.duration': {'phase': 1, 'name': 'Inicio'},
        'phase_2.Medicion.duration': {'phase': 2, 'name': 'Medicion'},
        'phase_3.Recirculacion.duration': {'phase': 3, 'name': 'Recirculacion'},
        'total_duration': {'phase': 4, 'name': 'Global'}
    }
    
    if texto in phase_mapping:
        return phase_mapping[texto][col_type]
    else:
        return 'None'

# Applying the function using apply to create the new columns
df['Phase'] = df['char_name'].apply(lambda x: nova_coluna(x, 'phase'))
df['Name'] = df['char_name'].apply(lambda x: nova_coluna(x, 'name'))

In [167]:
# df[(df['char_name'] =='phase_1.Inicio.duration')]

In [168]:
#Removing all the phases that are not in duration value
condicao = df['Phase'] == 'None'
df = df[~condicao]

In [169]:
#Reset index by rows
df.index = range(len(df.index))

In [170]:
#Renaming columns
df = df.rename(columns={'char_value': 'Duration_Sec', 'char_time':'Time', 'uniqueid': 'Batch'})

In [171]:
#Droppin char_name column
df = df.drop('char_name', axis=1)

In [172]:
#Organizing the columns of the DataFrame
df = df[['Batch', 'Phase', 'Name', 'Time' ,'Duration_Sec']]

In [173]:
def convert_min(sec):
    segundos_float = sec
    segundos = int(segundos_float)
    minutos = segundos // 60  # Divide to get the minutes
    horas = minutos // 60  # Calculate the hours
    minutos %= 60  # Calculate the remaining minutes after division by 60
    segundos_restantes = segundos % 60  # Calcula os segundos restantes


    # Formatting for the format 00:00.
    tempo_formatado = '{:02d}:{:02d}:{:02d}'.format(horas, minutos, segundos_restantes)
    
    return tempo_formatado

# Applying the function using apply to create the new columns of minutes
df['Duration_Min'] = df['Duration_Sec'].apply(convert_min)

In [174]:
df

,Batch,Phase,Name,Time,Duration_Sec,Duration_Min
0,44168,0,Operating,2023-02-11 10:19:25,3560.0,00:59:20
1,44168,0,Operating,2023-02-11 20:49:30,300.0,00:05:00
2,44168,0,Operating,2023-02-12 04:28:05,190.0,00:03:10
3,44168,0,Operating,2023-02-12 07:41:10,140.0,00:02:20
4,44168,0,Operating,2023-02-12 15:19:00,640.0,00:10:40
...,...,...,...,...,...,...
1824,47268,0,Operating,2023-03-31 11:34:00,155.0,00:02:35
1825,47268,1,Inicio,2023-03-31 11:36:35,115.0,00:01:55
1826,47268,2,Medicion,2023-03-31 11:38:30,10120.0,02:48:40
1827,47268,3,Recirculacion,2023-03-31 14:27:10,310.0,00:05:10


In [175]:
df.to_csv('dashboard_batch_cycle_time.csv', index=False)